In [1]:
#!!!The data file Gungor_2018_VictorianAuthorAttribution_data-train.csv can be obtained from https://archive.ics.uci.edu/ml/machine-learning-databases/00454/

##input data
import pandas as pd

#note: use your own path
path_to_datafile = '..//..//DS7004//u1720146_DS7004_courseworkCodeAndData//preparationWorks//fromDS7003_Gungor2018VictorianAuthorAttribution_NGram//Gungor_2018_VictorianAuthorAttribution_data-train.csv'
pathToGungorVict = path_to_datafile
gungorVictRow = pd.read_csv(pathToGungorVict, encoding = 'ISO-8859-1')

In [2]:
##form training data (2500 lines x 3) and test data (20 x 3)
##each line about 1000 words
#Use three authors' data:
#author:8 Charles Dickens total lines: 6914/ 14 George Eliot 2696/ 26 Jane Austen 4441
#each first 2500 lines for training, last 20 lines for testing. Each line has 1000 words
for i in [14, 26, 8]:
    allLines = gungorVictRow.loc[gungorVictRow['author'] == i]
    lines2500 = allLines.iloc[0:2500]
    linesLast20 = allLines.iloc[-20:]
    try:
        train = train.append(lines2500)
        test = test.append(linesLast20)
    except:
        train = lines2500
        test = linesLast20
train = train.sample(frac=1, random_state=42).reset_index(drop = True) #7500 lines suffled
test = test.sample(frac=1, random_state=42).reset_index(drop = True) #60 lines suffled

In [3]:
## Import various modules for forming a string cleaning function
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def text_to_wordlist( text, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    text = BeautifulSoup(text).get_text()
    #  
    # 2. Remove non-letters
    text = re.sub("[^a-zA-Z]"," ", text)
    #
    # 3. Convert words to lower case and split them
    words = text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:  #These three lines will not be used. Pleasesee the second parameter of this function
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [4]:
## Download the punkt tokenizer and form a sentence splitting function
import nltk.data
#nltk.download() #no need to use this line again after it has been used once  
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a text into parsed sentences
def text_to_sentences( text, tokenizer, remove_stopwords=False ):
    # Function to split a text into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(text.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call text_to_wordlist to get a list of words
            sentences.append( text_to_wordlist( raw_sentence,               remove_stopwords )) #defined as false in text_to_wordlist
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

#function for parsing the training set
def parsing_sentence_set(text_df):
    sentences = []  # Initialize an empty list of sentences

    print("Parsing sentences from training set")
    for text in text_df["text"]:
        sentences += text_to_sentences(text, tokenizer)
    return sentences

In [5]:
##use the functions to form a cleaned unlabelled training set
##for performming unsupervised learning
sentences = parsing_sentence_set(train)

Parsing sentences from training set


In [6]:
## Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',    level=logging.INFO)

# Set values for the single neural network layer's various parameters
#num_features = 300    # Word vector dimensionality                      
#min_word_count = 40   # Minimum word count                        
#num_workers = 4       # Number of threads to run in parallel
#context = 10          # Context window size
#downsampling = 1e-3   # Downsample setting for frequent words

num_features = 300    # Word vector dimensionality                      
min_word_count = 5    # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 6           # Context window size         
downsampling = 1e-3   # Downsample setting for frequent words
epochs= 20             #number of epochs

In [7]:
## Initialize and train the model (this will take some time)
# need to install gensim's word2vec
from gensim.models import word2vec
def form_model_from_sentences(sentences):
    print("Training model...")
    model = word2vec.Word2Vec(sentences, workers=num_workers, \
                size=num_features, min_count = min_word_count,\
                window = context, sample = downsampling, iter = epochs)

    # If you don't plan to train the model any further, calling 
    # init_sims will make the model much more memory-efficient.
    model.init_sims(replace=True)
    return model

In [8]:
##form the word2vec model with the training set which will be
##used in the following two methods:
##vector averaging and vector clustering of stop words
model = form_model_from_sentences(sentences)

2020-08-24 02:31:39,409 : INFO : collecting all words and their counts
2020-08-24 02:31:39,410 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model...


2020-08-24 02:31:40,509 : INFO : collected 9976 word types from a corpus of 7435641 raw words and 7500 sentences
2020-08-24 02:31:40,509 : INFO : Loading a fresh vocabulary
2020-08-24 02:31:40,525 : INFO : effective_min_count=5 retains 9940 unique words (99% of original 9976, drops 36)
2020-08-24 02:31:40,526 : INFO : effective_min_count=5 leaves 7435530 word corpus (99% of original 7435641, drops 111)
2020-08-24 02:31:40,553 : INFO : deleting the raw counts dictionary of 9976 items
2020-08-24 02:31:40,553 : INFO : sample=0.001 downsamples 55 most-common words
2020-08-24 02:31:40,554 : INFO : downsampling leaves estimated 5250158 word corpus (70.6% of prior 7435530)
2020-08-24 02:31:40,582 : INFO : estimated required memory for 9940 words and 300 dimensions: 28826000 bytes
2020-08-24 02:31:40,582 : INFO : resetting layer weights
2020-08-24 02:31:42,249 : INFO : training model with 4 workers on 9940 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=6
2020-08-24

2020-08-24 02:32:15,264 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-24 02:32:15,265 : INFO : EPOCH - 8 : training on 7435641 raw words (5249035 effective words) took 4.3s, 1208587 effective words/s
2020-08-24 02:32:16,276 : INFO : EPOCH 9 - PROGRESS: at 23.20% examples, 1205408 words/s, in_qsize 7, out_qsize 0
2020-08-24 02:32:17,282 : INFO : EPOCH 9 - PROGRESS: at 46.67% examples, 1215283 words/s, in_qsize 7, out_qsize 0
2020-08-24 02:32:18,283 : INFO : EPOCH 9 - PROGRESS: at 69.47% examples, 1208501 words/s, in_qsize 7, out_qsize 0
2020-08-24 02:32:19,287 : INFO : EPOCH 9 - PROGRESS: at 91.47% examples, 1194501 words/s, in_qsize 7, out_qsize 0
2020-08-24 02:32:19,629 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-08-24 02:32:19,634 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-24 02:32:19,645 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-24 02:32:19,648 : INFO : worker th

2020-08-24 02:32:55,533 : INFO : EPOCH - 16 : training on 7435641 raw words (5251264 effective words) took 5.7s, 918173 effective words/s
2020-08-24 02:32:56,537 : INFO : EPOCH 17 - PROGRESS: at 22.53% examples, 1180271 words/s, in_qsize 7, out_qsize 0
2020-08-24 02:32:57,538 : INFO : EPOCH 17 - PROGRESS: at 46.53% examples, 1218530 words/s, in_qsize 7, out_qsize 0
2020-08-24 02:32:58,546 : INFO : EPOCH 17 - PROGRESS: at 70.67% examples, 1231328 words/s, in_qsize 7, out_qsize 0
2020-08-24 02:32:59,552 : INFO : EPOCH 17 - PROGRESS: at 93.87% examples, 1226001 words/s, in_qsize 7, out_qsize 0
2020-08-24 02:32:59,832 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-08-24 02:32:59,844 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-24 02:32:59,861 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-24 02:32:59,863 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-24 02:32:59,864 : INFO : EPOCH

In [9]:
##check the model
# king - man + woman = queen?
print(model.most_similar(positive=['king', 'woman'], negative=['man']))

[('queen', 0.3741656541824341), ('saxon', 0.31361255049705505), ('throne', 0.3131310045719147), ('conqueror', 0.31052806973457336), ('earl', 0.29378655552864075), ('girl', 0.28226879239082336), ('child', 0.2698107063770294), ('dying', 0.26731109619140625), ('reign', 0.26275989413261414), ('pillow', 0.2603800594806671)]


<ipython-input-9-e2296c31c599>:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  print(model.most_similar(positive=['king', 'woman'], negative=['man']))


In [13]:
##check the model
# king - man + woman = queen?
print(model.most_similar(positive=['better', 'bad'], negative=['good']))

[('worse', 0.3971032500267029), ('wiser', 0.37148696184158325), ('sooner', 0.35369372367858887), ('happier', 0.3022949993610382), ('easier', 0.27671748399734497), ('fairer', 0.26694223284721375), ('bigger', 0.2663915753364563), ('safer', 0.2650602161884308), ('apprehended', 0.2614726722240448), ('dearer', 0.2573417127132416)]


<ipython-input-13-2a4fea38c5af>:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  print(model.most_similar(positive=['better', 'bad'], negative=['good']))


In [17]:
# ****************************************************************
##first method: vector averaging of stop words:
import gensim
all_stopwords = set(gensim.parsing.preprocessing.STOPWORDS)

#be careful: nword and counter must be integers --Chiu
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph which are stop words
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    index2word_set2 = all_stopwords
    #
    # Loop over each word in the text and, if it is in the model's
    # vocaublary and is a stop word add its feature vector to the total
    for word in words:
        if word in index2word_set: #and word in index2word_set2: 
            if word in index2word_set2:
                nwords = nwords + 1
                featureVec = np.add(featureVec, model[word])
    # 
    # Divide the result by the number of words to get the average
    if nwords == 0:
        nwords = 1 #avoid devided by zero (i.e. no stop word)
    featureVec = np.divide(featureVec,nwords)
    return featureVec


In [18]:
def getAvgFeatureVecs(texts, model, num_features):
    # Given a set of texts (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    textFeatureVecs = np.zeros((len(texts),num_features),dtype="float32")
    # 
    # Loop through the texts
    for text in texts:
       #
       # Print a status message every 100th text
        if counter%100 == 0:
            haha = counter; hihi = len(texts)
            print(f"Text {haha} of {hihi}") #% (counter, len(texts))
       # 
       # Call the function (defined above) that makes average feature vectors
        #textFeatureVecs[counter] = makeFeatureVec(text, model, num_features)
        textFeatureVecs[counter] = makeFeatureVec(text, model, num_features)
       # Increment the counter
        counter = counter + 1
    return textFeatureVecs

In [19]:
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. 

clean_train_texts = []
for text in train["text"]:
    #clean_train_reviews.append( review_to_wordlist( review, \
        #remove_stopwords=True )) #do not remove stop words
    clean_train_texts.append( text_to_wordlist( text ))

trainDataVecs = getAvgFeatureVecs( clean_train_texts, model, num_features )

print("Creating average feature vecs for test texts")
clean_test_texts = []
for text in test["text"]:
    #clean_test_texts.append( text_to_wordlist( review, remove_stopwords=True ))
    clean_test_texts.append( text_to_wordlist( text ))

testDataVecs = getAvgFeatureVecs( clean_test_texts, model, num_features )

# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs, train["author"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"true_author":test["author"], "pred_author":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

Text 0 of 7500


<ipython-input-17-7a901c89f86d>:29: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  featureVec = np.add(featureVec, model[word])


Text 100 of 7500
Text 200 of 7500
Text 300 of 7500
Text 400 of 7500
Text 500 of 7500
Text 600 of 7500
Text 700 of 7500
Text 800 of 7500
Text 900 of 7500
Text 1000 of 7500
Text 1100 of 7500
Text 1200 of 7500
Text 1300 of 7500
Text 1400 of 7500
Text 1500 of 7500
Text 1600 of 7500
Text 1700 of 7500
Text 1800 of 7500
Text 1900 of 7500
Text 2000 of 7500
Text 2100 of 7500
Text 2200 of 7500
Text 2300 of 7500
Text 2400 of 7500
Text 2500 of 7500
Text 2600 of 7500
Text 2700 of 7500
Text 2800 of 7500
Text 2900 of 7500
Text 3000 of 7500
Text 3100 of 7500
Text 3200 of 7500
Text 3300 of 7500
Text 3400 of 7500
Text 3500 of 7500
Text 3600 of 7500
Text 3700 of 7500
Text 3800 of 7500
Text 3900 of 7500
Text 4000 of 7500
Text 4100 of 7500
Text 4200 of 7500
Text 4300 of 7500
Text 4400 of 7500
Text 4500 of 7500
Text 4600 of 7500
Text 4700 of 7500
Text 4800 of 7500
Text 4900 of 7500
Text 5000 of 7500
Text 5100 of 7500
Text 5200 of 7500
Text 5300 of 7500
Text 5400 of 7500
Text 5500 of 7500
Text 5600 of 7500
T

In [30]:
confusion_matrix = pd.crosstab(output['true_author'], output['pred_author'], rownames=['Actural'], colnames=['Predicted'])
print('Confusion matrix:\n', confusion_matrix)

Confusion matrix:
 Predicted  8   14  26
Actural              
8          18   2   0
14          2  18   0
26          0   0  20


In [31]:
from sklearn import metrics
print('Accuracy: ', metrics.accuracy_score(output['true_author'], output['pred_author']))

Accuracy:  0.9333333333333333


In [32]:
print('f1 score: ', metrics.f1_score(output['true_author'], output['pred_author'], average = 'weighted'))

f1 score:  0.9333333333333333


In [33]:
# ****************************************************************
##second method: vector clustering of stop words (use KMeans):

from sklearn.cluster import KMeans
import time

start = time.time() # Start time (several to tens of minutes)

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.syn0
num_clusters = word_vectors.shape[0] / 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = int(num_clusters) )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print("Time taken for K Means clustering: ", elapsed, "seconds.")

# Create a Word / Index dictionary, mapping each vocabulary word to
#a cluster number

word_centroid_map = dict(zip( model.wv.index2word, idx ))

# For the first 10 clusters
for cluster in range(0,10):
    #
    # Print the cluster number  
    #print "\nCluster %d" #% cluster
    print(f"\nCluster {cluster}")
    #
    # Find all of the words for that cluster number, and print them out
    a_view = word_centroid_map.items()
    tuples = list(a_view)
    words = []
    for i in range(0,len(word_centroid_map.values())):
        if( tuples[i][1] == cluster ):
            words.append(tuples[i][0])
    print(words)

<ipython-input-33-9091e8a1385b>:11: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  word_vectors = model.wv.syn0


Time taken for K Means clustering:  527.5698883533478 seconds.

Cluster 0
['stroke', 'ripe', 'tain', 'shaggy', 'comb', 'growling', 'brim', 'skull', 'goat']

Cluster 1
['divine', 'demon', 'eternal', 'salvation', 'pathos', 'symbol']

Cluster 2
['pain', 'pity', 'secret', 'parting', 'shame', 'shock', 'horrible', 'pang']

Cluster 3
['bv', 'ana', 'wan', 'til', 'rf', 'fu', 'ally', 'tp', 'arid', 'sow', 'js', 'bait']

Cluster 4
['stone', 'prison', 'walls', 'pictures', 'size', 'wheels', 'rainbow', 'maps']

Cluster 5
['numbers', 'majority']

Cluster 6
['restored', 'reduced']

Cluster 7
['interesting', 'inferior', 'amusing', 'musical', 'unsatisfactory']

Cluster 8
['hers', 'faults', 'champion']

Cluster 9
['reach', 'pointed', 'stretch', 'stretching', 'shaped', 'lays', 'carries', 'tracing', 'stretches', 'straightened']


In [34]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map and word in all_stopwords:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [35]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["text"].size, int(num_clusters)),     dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for text in clean_train_texts:
    train_centroids[counter] = create_bag_of_centroids( text,         word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros((test["text"].size, int(num_clusters)),     dtype="float32" )

counter = 0
for text in clean_test_texts:
    test_centroids[counter] = create_bag_of_centroids( text,         word_centroid_map )
    counter += 1


In [36]:
# This cell take some minutes
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train["author"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"true_author":test["author"], "pred_author":result})
output.to_csv( "BagOfCentroidsAuthor.csv", index=False, quoting=3 )


Fitting a random forest to labeled training data...


In [37]:
confusion_matrix = pd.crosstab(output['true_author'], output['pred_author'], rownames=['Actural'], colnames=['Predicted'])
print('Confusion matrix:\n', confusion_matrix)

Confusion matrix:
 Predicted  8   14  26
Actural              
8          19   1   0
14          1  19   0
26          0   0  20


In [38]:
from sklearn import metrics
print('Accuracy: ', metrics.accuracy_score(output['true_author'], output['pred_author']))

Accuracy:  0.9666666666666667


In [39]:
print('f1 score: ', metrics.f1_score(output['true_author'], output['pred_author'], average = 'weighted'))

f1 score:  0.9666666666666667
